In [21]:
BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': [
                'привет',
                'здаров',
                'день добрый'
                ]
            },
        'bot': {
            'examples':[
                'бот',
                'робот'
            ]
        },
        'goodbye': {
            'examples': [
                'пака',
                'пока',
                'давай',
                'счастливо'
                'удачи'
            ]
        }
    }
}

In [22]:
with open('..\dialogues.txt', encoding='utf-8') as f:
    data = f.read()

In [23]:
dialogues = []
for d in data.split('\n\n'):
    replicas = []
    for t in d.split('\n')[:2]:    # первые две части строки
        t = t[2:].lower()
        replicas.append(t)
    if len(replicas) == 2 and 5<len(replicas[0])<25 and 5<len(replicas[1])<25:
        dialogues.append(replicas)    


In [24]:
BOT_DIALOGUES = dialogues[:100000]

In [25]:
CLASS_THRESHOLD = 0.3
GENER_THRESHOLD = 0.7

In [26]:
X_text = []
y = []

for intent, value in BOT_CONFIG['intents'].items():
    for example in value['examples']:
        X_text.append(example)
        y.append(intent)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.metrics.distance import edit_distance

In [28]:
VC = CountVectorizer()
X =VC.fit_transform(X_text)

In [29]:
CLF = LogisticRegression()
CLF.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
def get_intent(text):
    probas = CLF.predict_proba(VC.transform([text]))
    max_value = max(probas[0])
    if max_value > CLASS_THRESHOLD:
        idx = probas[0].tolist().index(max_value)
        return CLF.classes_[idx]

In [48]:
def get_answer_by_gen(text):
    low_text = text.lower()
    for question, answer in BOT_DIALOGUES:
        if abs(len(question) - len(low_text)) / len(question) < 1 - GENER_THRESHOLD:
            dist = edit_distance(question, low_text)
            l = len(question)
            diff = dist / l
            sim = 1 - diff
            if sim > GENER_THRESHOLD:
                return answer
    return None

In [52]:
get_answer_by_gen('досвидания')

'вы ко мне?'